## Importing our data
We use the ImageDataGenerator of Tensorflow to make the training, validation and testing sets

In [1]:
import os
import pandas
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen=ImageDataGenerator( rescale=1./255,rotation_range=0.2, shear_range=0.2,
                                  zoom_range=0.2, width_shift_range=0.2,
                                  height_shift_range=0.2, validation_split=0.2)
train_data=train_datagen.flow_from_directory('./prepared_data/train_data',target_size=(300,300),batch_size=80,class_mode='categorical',subset='training')
val_data=train_datagen.flow_from_directory('./prepared_data/train_data',target_size=(300,300),batch_size=80,class_mode='categorical',subset='validation')

test_datagen=ImageDataGenerator( rescale=1./255,rotation_range=0.2, shear_range=0.2,
                                  zoom_range=0.2, width_shift_range=0.2,
                                  height_shift_range=0.2)

test_data=test_datagen.flow_from_directory('./prepared_data/test_data',target_size=(300,300),batch_size=80,class_mode='categorical')

Found 11728 images belonging to 5 classes.
Found 2929 images belonging to 5 classes.
Found 3665 images belonging to 5 classes.


In [3]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten
from tensorflow.keras.applications import InceptionV3

## Making our Model
We make use of the InceptionV3 model in making our model. Because the imagenet weights were not trained on images related to our use case, we train it from scratch

In [4]:
base_model = InceptionV3(include_top = False, weights = None, input_tensor = Input(shape = (300,300,3)))
head_model = base_model.output

In [5]:
head_model=Flatten()(head_model) # flatten the incoming image data to a 1d vector
head_model=Dense(64,activation='relu')(head_model)
head_model=Dropout(0.2)(head_model) # avoids overfitting
head_model=Dense(5, activation='softmax')(head_model) # gives the class

model = Model(inputs = base_model.input, outputs = head_model)

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint('./model checkpoint/model.keras', monitor = 'val_accuracy', save_best_only = True)
earlystop = EarlyStopping(monitor = 'val_accuracy', patience = 7, restore_best_weights = True)
learning_rate = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 3)
callbacks = [checkpoint, earlystop, learning_rate]

In [7]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_data, validation_data = val_data, 
          epochs = 30, 
          callbacks = callbacks)

Epoch 1/30


D:\Dataset\Diabetic-Retinopathy\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


147/147 ━━━━━━━━━━━━━━━━━━━━ 1839s 12s/step - accuracy: 0.6793 - loss: 4.9457 - val_accuracy: 0.7344 - val_loss: 0.9775 - learning_rate: 0.0010
Epoch 2/30
134/147 ━━━━━━━━━━━━━━━━━━━━ 2:44 13s/step - accuracy: 0.7287 - loss: 0.9481

KeyboardInterrupt: 

In [ ]:
loss_train, acc_train = model.evaluate(train_data)
print(acc_train)
print(loss_train)

In [ ]:
loss_val, acc_val = model.evaluate(val_data)
print(acc_val)
print(loss_val)

In [ ]:
loss_test, acc_test = model.evaluate(test_data)
print(acc_test)
print(loss_test)

In [ ]:
y_pred = model.predict(test_data)
y_pred

In [8]:
pip uninstall tensorflow
pip install tensorflow-gpu


SyntaxError: invalid syntax (4212133585.py, line 1)